installation PostgreSQL
Installation du library psycopg

In [72]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np
import psycopg


In [73]:
conversation_file_path = "../data/conversation.txt"
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
db_connection_str = "dbname=chatbot_rag user=postgres password=Admin@1234 host=localhost port=5432"

In [74]:
load_dotenv()  # charge .env

def create_conversation_list(file_path:str)->list[str]:
    with open(file_path, encoding="UTF-8") as file:
        text= file.read()
        text_list= text.split("\n")
        filtered_list = [chaine.removeprefix("     ") for chaine in text_list if not chaine.startswith("<") ]
        
        return filtered_list

def calculate_embedding(corpus: str, client: OpenAI) -> list[float]:
    embeddings=client.embeddings.create(
        model="text-embedding-ada-002",
        input="The food was delicious and the waiter...",
        
    ).data
    return embeddings[0].embedding

def save_embeding(corpus: str, embedding: list[float], cursor) -> None:
     cursor.execute("""
            INSERT INTO embeddings (corpus, embedding)
            VALUES (%s, %s)
        """, (corpus, embedding))
     
def retrieve_similar_corpus(input_corpus: str, client: OpenAI, db_connection_str: str) -> tuple[int, str, list[float]]:
    input_corpus_embedding = calculate_embedding(corpus=input_corpus, client=client)
    with psycopg.connect(db_connection_str) as conn:
        with conn.cursor() as cur:
            query = """
            SELECT id, corpus, embedding FROM embeddings
            ORDER BY embedding <=> %s ::vector
            LIMIT 1;
            """
            cur.execute(query, [input_corpus_embedding])
            result = cur.fetchone()
            return result

def generate_response(input_corpus: str, client: OpenAI=openai_client, db_connection_str: str= db_connection_str):
    similar_text = retrieve_similar_corpus(input_corpus=input_corpus, client=client, db_connection_str=db_connection_str)[1 ]
    completion = openai_client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a helpful assistant working for a loan company. Your role is to assist clients in understanding the website, explain the different loan services offered, guide them through the application process, and answer any questions they may have to ensure a smooth and clear experience. You should retrieve information from the company’s database and reformulate the responses in a clear and precise manner to make it easy for clients to understand."
            },
            {
                "role": "user",
                "content": similar_text
            }
        ]
    )
    return completion.choices[0].message.content
   
    
    

In [75]:
# Connexion à la base
with psycopg.connect(db_connection_str) as conn:
    with conn.cursor() as cur:
        cur.execute("""Drop table if exists embeddings;""")
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        # Création table
        cur.execute("""
        CREATE TABLE IF NOT EXISTS embeddings (
           id SERIAL PRIMARY KEY,
           corpus TEXT NOT NULL,
           embedding VECTOR(1536)
        );
        """)

        # Données
        corpus_list= create_conversation_list(file_path=conversation_file_path)
        for corpus in corpus_list:
            embedding = calculate_embedding(corpus=corpus, client=openai_client)
            save_embeding(corpus=corpus, embedding=embedding, cursor=cur)

    # Validation transaction
    conn.commit()


Call "C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\Build\vcvars64.bat"

In [76]:
user_message = "vous m'avez dit de rappeler"
retrieve_similar_corpus(input_corpus=user_message, client=openai_client, db_connection_str=db_connection_str)

(2,
 "c: oui bonjour e j'ai appel� ce matin pour avoir e monsieur Nom vous m'avez dit de rappeler",
 '[0.0023165925,-0.009348879,0.015702054,-0.0076987036,-0.0046903063,0.014927741,-0.009862972,-0.03820791,-0.006930737,-0.028636891,0.02526038,0.01815193,-0.003614519,-0.025552332,0.0005164732,-0.016374819,0.028230695,0.0053916313,0.009678914,-0.016425593,-0.015397407,0.004271416,0.0070259394,-0.0072036507,-0.0038779124,0.018545434,0.00874593,-0.022670873,0.011481414,0.023965625,0.015625892,-0.0034939293,-0.034932945,-0.004201601,-0.026174322,-0.02151575,-0.0057375333,0.01172894,0.008422242,0.004134959,0.019180117,-0.014458076,0.009012497,0.006378563,-0.04567178,0.017898057,-0.005582036,-0.0007247285,-0.02211235,-0.003906473,0.021071471,-0.017618796,-0.01172894,-0.02259471,0.016349431,0.01717452,-0.008371467,0.0015763936,0.025044587,-0.025006505,0.007831987,0.0058327354,-0.022188513,0.0029036743,-0.0061310367,-0.025565026,-0.008054126,0.0010416732,0.00022134566,0.004626838,0.020728743,0.

In [77]:
generate_response(input_corpus=user_message)

'Bonjour ! Comment puis-je vous aider aujourd’hui ?\n\nJe peux vous accompagner pour:\n- Informations sur nos prêts (personnel, auto, immobilier, rachat de crédit)\n- Simulation de mensualités et taux\n- Pièces à fournir et critères d’éligibilité\n- Suivi d’une demande en cours\n\nSouhaitez-vous continuer en français ou en anglais ?'